# Getting the most solar power for your dollar
## Exploratory Data Analysis
### Zachary Brown

Now that the raw data has been wrangled into a useful dataframe it is time to explore the data and identify correlations and trends that may be important for modeling. 

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import statsmodels.api as sm 
from statsmodels.graphics.api import abline_plot
from sklearn.model_selection import train_test_split
sns.set_theme('notebook')

AttributeError: module 'seaborn' has no attribute 'set_theme'

In [ ]:
print(os.getcwd())
os.chdir(r"..\data\interim")
print(os.getcwd())

In [ ]:
data = pd.read_csv('wrangled_data.csv', index_col=0, low_memory=False)
data.shape

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
col = data.columns.to_series().groupby(data.dtypes).groups
print(col)

In [ ]:
# Year was read in as a float, so I'm going to change that to integer

data['year'] = data['year'].astype(int)

As a first step in this analysis I'm going to take a look at the distribution of the price per KW values for the entire dataset.

In [ ]:
sns.boxplot(x = data['price_per_kw'], showfliers = False)
plt.title('Cost Efficiency Distribution')
plt.xlabel('Price per KW')
plt.show()

In [ ]:
data['price_per_kw'].describe()

One useful tool to quickly identify a difference in distributions is the empirical cumulative distribution function (ECDF). I'd like to determine whether there are any clear differences in prices per KW in TX versus the rest of the US, so I will first create a function to quickly produce an EDCF for a given column. Then I'll create two new dataframes, one with just TX data, the other with all other states. Then I'll overlay the ECDFs of both to compare. 

In [ ]:
def ecdf(column):
    n = len(column)

    # x-data for the ECDF: x
    x = np.sort(column)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

In [ ]:
# Creating the two new dataframes
tx = data[data['state_TX'] == 1]
not_tx = data[data['state_TX'] == 0]

In [ ]:
# Plotting the two ECDFs against one another

tx_x, tx_y = ecdf(tx['price_per_kw'])
us_x, us_y = ecdf(not_tx['price_per_kw'])

plt.plot(tx_x, tx_y, color='r')
plt.plot(us_x, us_y, color='b')
plt.xlim(0,10000)
plt.title('Cost Efficiency Distribution: Texas vs Rest of the US')
plt.xlabel('Price per KW')
plt.ylabel('ECDF')
plt.legend(['Texas', 'Rest of US'], loc='lower right')
plt.show()

Very interesting! Although the rest of the US has some installations that are more cost efficient than those in Texas, the Texas installations as a whole tend to be more cost efficient than the rest of the country. It seems likely that this will factor into the model later on. Just to confirm whether these results are significantly different, I'll run a two-tailed t-test with the null hypothesis that the mean price per KW of Texas installations and the rest of the US are the same. Before performing the t-test I need to confirm that the data are normally distributed, so I'll perform Shapiro-Wilk tests for normality first. If those results are greater than 0.05 the data is assumed to be normal and I can proceed with the t-test.

In [ ]:
scipy.stats.shapiro(tx['price_per_kw'])

In [ ]:
scipy.stats.shapiro(not_tx['price_per_kw'])

Ok, both sets of data pass the Shapiro-Wilk test, so now I'll run the t-test.

In [ ]:
scipy.stats.ttest_ind(tx['price_per_kw'], not_tx['price_per_kw'])

Ok, our t-test confirms that there is a real difference in average cost efficiency between Texas and the rest of the US. I'll determine the Pearson correlation coefficient to quantify how linear the relationship is.

In [ ]:
scipy.stats.pearsonr(data['state_TX'], data['price_per_kw'])

So there is a -0.018 correlation coefficient between whether the installation is in Texas and the installation cost efficiency. That's good to know!  I'll keep an eye out for that to show up in the model I generate later.

It seems very possible that pricing may vary throughout the year given seasonal changes in weather. For example one might expect an increase in installations, and hence an increase in price, during the winter or spring so that the panels are ready in time for summer. I'll prepare a series of boxplots of the price per KW broken down by month to get a quick look at whether there is any month to month effect.

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

sns.boxplot(x='month', y='price_per_kw', data=data, palette='colorblind', showfliers=False)
plt.title('Cost Efficiency by Installation Month')
plt.xlabel('Month')
plt.xticks(np.arange(12), months)
plt.ylabel('Price per KW')
plt.show()

In [ ]:
chi2_month = data[['month', 'price_per_kw']].copy()
chi2_month = chi2_month[chi2_month['month'] >= 0]
chi2_month = chi2_month[chi2_month['price_per_kw'] >= 0]
scipy.stats.chi2_contingency(chi2_month)

Although there isn't a large obvious difference upon visual inspection of the box plots, the chi squared test suggests that there is a significant correlation between the installation month and price efficiency.

Next I want to check for quick correlations in the categorical columns such as 'expansion_system', 'multiple_phase_system', etc. To do so I'll create a loop to automatically make box plots for each.

In [ ]:
for col in data.columns:
    if data.dtypes[col] == 'int64' and 'state' not in col and 2 not in data[col].unique():
        sns.boxplot(x=col, y='price_per_kw', data=data, showfliers=False)
        plt.title(''.join(['Imapact of ', col.title().replace('_', " "), ' on Cost Efficiency']))
        plt.xlabel(col.title().replace('_', " "))
        plt.ylabel('Price per KW')
        plt.show()    

While there aren't any plots that show an obvious dramatic difference between their yes (1) and no (0) values, 'self_installed' and 'additional_modules' appear to improve cost efficiency, while 'bipv_module_1', 'additional_inverters', and 'solar_storage_hybrid_inverter_1' all appear to reduce cost efficiency. These may prove important to model development.

I'll go through each of those columns, remove the missing data, split the dataframes by the response to the property of interest, and then perform t-tests to identify whether the mean price per KW is different based on that property. If the t-test statistic is significant I'll then perform a Pearson correlation test to identify the correlation coefficient.

In [ ]:
# self_installed
si_non_null = data[data['self_installed'] != -1]
si_yes = si_non_null[si_non_null['self_installed'] == 1]
si_no = si_non_null[si_non_null['self_installed'] == 0]
print(scipy.stats.ttest_ind(si_yes['price_per_kw'], si_no['price_per_kw']))

In [ ]:
scipy.stats.pearsonr(data['self_installed'], data['price_per_kw'])

In [ ]:
# additional_modules

am_non_null = data[data['additional_modules'] != -1]
am_yes = am_non_null[am_non_null['additional_modules'] == 1]
am_no = am_non_null[am_non_null['additional_modules'] == 0]
print(scipy.stats.ttest_ind(am_yes['price_per_kw'], am_no['price_per_kw']))

In [ ]:
scipy.stats.pearsonr(data['additional_modules'], data['price_per_kw'])

In [ ]:
# bipv_module_1 - BIPV stands for building integrated photovoltaic, where the solar panels are designed to blend in with 
# the rest of the building design.

bm1_non_null = data[data['bipv_module_1'] != -1]
bm1_yes = bm1_non_null[bm1_non_null['bipv_module_1'] == 1]
bm1_no = bm1_non_null[bm1_non_null['bipv_module_1'] == 0]
print(scipy.stats.ttest_ind(bm1_yes['price_per_kw'], bm1_no['price_per_kw']))

In [ ]:
scipy.stats.pearsonr(data['bipv_module_1'], data['price_per_kw'])

In [ ]:
# additional_inverters

ai_non_null = data[data['additional_inverters'] != -1]
ai_yes = ai_non_null[ai_non_null['additional_inverters'] == 1]
ai_no = ai_non_null[ai_non_null['additional_inverters'] == 0]
print(scipy.stats.ttest_ind(ai_yes['price_per_kw'], ai_no['price_per_kw']))

In [ ]:
scipy.stats.pearsonr(data['additional_inverters'], data['price_per_kw'])

In [ ]:
# solar_storage_hybrid_inverter_1

sshi1_non_null = data[data['solar_storage_hybrid_inverter_1'] != -1]
sshi1_yes = sshi1_non_null[sshi1_non_null['solar_storage_hybrid_inverter_1'] == 1]
sshi1_no = sshi1_non_null[sshi1_non_null['solar_storage_hybrid_inverter_1'] == 0]
print(scipy.stats.ttest_ind(sshi1_yes['price_per_kw'], sshi1_no['price_per_kw']))

In [ ]:
scipy.stats.pearsonr(data['solar_storage_hybrid_inverter_1'], data['price_per_kw'])

Great, all of these properties do appear to significantly impact the cost efficiency of the installation as I predicted based on the boxplots. 

Now I want to look for any correlations between the continuous variables and price per KW to see if those play any role here. I'm going to limit the y-axis to the 99.95th percentile of the data to cut out potential outliers which may hide the shape of the bulk data.

In [ ]:
for col in data.select_dtypes('float64').columns:
    if col != 'price_per_kw':
        sns.scatterplot(data=data, x=col, y='price_per_kw', alpha=0.1)
        plt.title(''.join(['Imapact of ', col.title().replace('_', " "), ' on Cost Efficiency']))
        plt.xlabel(col.title().replace('_', " "))
        plt.ylabel('Price per KW')
        plt.ylim(0, np.quantile(data['price_per_kw'], 0.9995))
        plt.show()

These give an interesting quick look at the data. I want to dig into a few of them a little more. I'll start with efficiency_module_1 by removing missing values, creating an OLS regression of the data, and checking the R-squared value as well as the p-values for the parameters to determine whether the model is a good representation of the data. I'll then overlay the regression line over the data to visually assess the trend.

In [ ]:
# First I remove the missing data.
em1_no_null = data[data['efficiency_module_1'] != -1]

dep_em1 = em1_no_null[['price_per_kw']]
ind_em1 = sm.add_constant(em1_no_null[['efficiency_module_1']])

x_train, x_test, y_train, y_test = train_test_split(ind_em1, dep_em1, test_size=0.25)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Now I'll create the model and check the R-squared.
em1_model = sm.OLS(y_train,x_train)
em1_results = em1_model.fit()
em1_results.summary()

In [ ]:
# The R-squared is 0.000, which means the residuals are very large for this regression line. 
# Let's check the slope and intercept parameters to get an idea of their magnitudes.
b, m = em1_results.params
em1_results.params

In [ ]:
# Now I want to see if the parameters have significant p-values.
em1_results.pvalues

In [ ]:
scipy.stats.pearsonr(data['efficiency_module_1'], data['price_per_kw'])

In [ ]:
# I'll make the arrays for the regression line.
x=np.array([0, 0.23])
y=(m*x)+b

# And now plot the data with the regression line overlay.
sns.scatterplot(data=em1_no_null, x='efficiency_module_1', y='price_per_kw', alpha = 0.1, color='blue')
plt.plot(x, y, color='red')
plt.title('Cost Efficiency Correlation with PV Module Efficiency')
plt.ylim(0, np.quantile(em1_no_null['price_per_kw'], 0.9995))
plt.xlabel('Efficiency of PV Module')
plt.ylabel('Price per KW')
plt.show

So analysis of efficiency_module_1 shows that there is a linear correlation that is significant, however, the residuals are still very large, giving me an R-squared value of 0.000. This means that while the correlation is significant, it is not a good fit for the cost efficiency metric on its own. I'll perform the same analysis on nameplate_capacity_module_1 and inverter_loading_ratio to check those variables as well.

In [ ]:
# Nameplate capacity is the maximum output the system can generate
ncm1_no_null = data[data['nameplate_capacity_module_1'] != -1]
ncm1_no_null = ncm1_no_null.dropna(axis=0, subset=['nameplate_capacity_module_1', 'price_per_kw'])

dep_ncm1 = ncm1_no_null[['price_per_kw']]
ind_ncm1 = sm.add_constant(ncm1_no_null[['nameplate_capacity_module_1']])

X_train, X_test, y_train, y_test = train_test_split(ind_ncm1, dep_ncm1, test_size=0.25)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
ncm1_model = sm.OLS(y_train,X_train)
ncm1_results = ncm1_model.fit()
ncm1_results.summary()

In [ ]:
# Once again the R-squared is 0.000, which means the residuals are very large for this regression line. 
# Let's assign the slope and intercept parameters to variables so we can plot them later.
b, m = ncm1_results.params
ncm1_results.params

In [ ]:
# Now I want to see if the parameters have significant p-values.
ncm1_results.pvalues

In [ ]:
# I'll make the arrays for the regression line.
x=np.array([0, 500])
y=(m*x)+b

# And now plot the data with the regression line overlay.
sns.scatterplot(data=ncm1_no_null, x='nameplate_capacity_module_1', y='price_per_kw', alpha = 0.1, color='blue')
plt.plot(x, y, color='red')
plt.title('Cost Efficiency Correlation with Advertised Capacity')
plt.ylim(0, np.quantile(ncm1_no_null['price_per_kw'], 0.9995))
plt.xlabel('Advertised Photovoltaic Capacity')
plt.ylabel('Price per KW')
plt.show

This regression line not only appears flat and has an R-squared of 0.000, but also has a p-value for the slope of 0.86, suggesting that nameplate_capacity_module_1 is not significantly correlated to the price_per_kw. The last continuous feature that looks worth checking is inverter_loading_ratio.

In [ ]:
ilr_no_null = data[data['inverter_loading_ratio'] != -1]
ilr_no_null = ilr_no_null.dropna(axis=0, subset=['inverter_loading_ratio', 'price_per_kw'])

dep_ilr = ilr_no_null[['price_per_kw']]
ind_ilr = sm.add_constant(ilr_no_null[['inverter_loading_ratio']])

X_train, X_test, y_train, y_test = train_test_split(ind_ilr, dep_ilr, test_size=0.25)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
ilr_model = sm.OLS(y_train,X_train)
ilr_results = ilr_model.fit()
ilr_results.summary()

In [ ]:
# Once again the R-squared is 0.000, which means the residuals are very large for this regression line. 
# Let's assign the slope and intercept parameters to variables so we can plot them later.
b, m = ilr_results.params
ilr_results.params

In [ ]:
# Now I want to see if the parameters have significant p-values.
ilr_results.pvalues

In [ ]:
scipy.stats.pearsonr(data['inverter_loading_ratio'], data['price_per_kw'])

In [ ]:
# I'll make the arrays for the regression line.
x=np.array([0.7, 1.6])
y=(m*x)+b

# And now plot the data with the regression line overlay.
sns.scatterplot(data=ilr_no_null, x='inverter_loading_ratio', y='price_per_kw', alpha = 0.1, color='blue')
plt.plot(x, y, color='red')
plt.title('Cost Efficiency Correlation with Inverter Loading Ratio')
plt.ylim(0, np.quantile(ilr_no_null['price_per_kw'], 0.9995))
plt.xlabel('Inverter Loading Ratio')
plt.ylabel('Price per KW')
plt.show

Once again we have an R-squared of 0, so the inverter_loading_ratio doesn't predict the cost efficiency well on its own, however, the p-value for the slope of the regression line suggests that there is a significant correlation here that I may need to consider when building a model.

One of the last columns I haven't dealt with yet is the date of battery install variable. It's listed as an object type right now, so I'm going to have to check how it's formatted, change it to datetime, break it out to month and year, then check each for any meaningful correlations to cost efficiency.

In [ ]:
dobi = data[~data['date_of_battery_install'].isnull()]
print(dobi['date_of_battery_install'].head())

In [ ]:
dobi_c = dobi.copy()
dobi_c['date_of_battery_install'] = pd.to_datetime(dobi_c['date_of_battery_install'])
dobi_c['month_of_battery_install'] = dobi_c['date_of_battery_install'].dt.month
dobi_c['year_of_battery_install'] = dobi_c['date_of_battery_install'].dt.year

In [ ]:
sns.boxplot(x='month_of_battery_install', y='price_per_kw', data=dobi_c, palette='colorblind', showfliers=False)
plt.title('Cost Efficiency by Month of Battery Installation')
plt.xlabel('Battery Installation Month')
plt.ylabel('Price per KW')
plt.show()

In [ ]:
sns.boxplot(x='year_of_battery_install', y='price_per_kw', data=dobi_c, palette='colorblind', showfliers = False)
plt.title('Cost Efficiency by Year of Battery Installation')
plt.xlabel('Battery Installation Year')
plt.ylabel('Price per KW')
plt.show()

Now these are really interesting. It looks like we have more cost efficiency variation when looking at battery installation month than solar panel installation month, and even more interestingly, there are large differences in cost efficiency when split by the year of a battery installation. 

I'll perform chi-squared contingency tests on both to confirm whether the categories of each variable (month and year of battery install) are independent or not.

In [ ]:
chi_month = dobi_c[['month_of_battery_install', 'price_per_kw']].copy()
scipy.stats.chi2_contingency(chi_month)

In [ ]:
chi_year = dobi_c[['year_of_battery_install', 'price_per_kw']].copy()
scipy.stats.chi2_contingency(chi_year)

In both cases we get very large chi-squared values and p-values of 0.0, meaning that the month and year of battery installation do have an impact on cost efficiency of the solar panel installation.

I'll need to export the updated dataframe for modeling to make sure I capture those broken out date features in the model.

In [ ]:
print(os.getcwd())
os.chdir(r"..\processed")
print(os.getcwd())

### Conclusions
As a final wrap-up of this analysis I'll summarize the findings below.

The Pearson correlation coefficients for categorical features with statistically significant impact on cost efficiency are listed below:

* state_TX - -0.018 
* self_installed: -0.014 
* additional_modules: 0.018 
* bipv_module_1: 0.017 
* additional_inverters: 0.020 
* solar_storage_hybrid_inverter_1: 0.031 

Although none of the continuous features had an R-squared above 0.000, two features both had significant p-values for their regression line slopes, suggesting that they do have some correlation with price_per_kw. They're listed below with their Pearson correlation coefficients:

* efficiency_module_1: 0.013
* inverter_loading_ratio: 0.005

All of these correlation coefficients suggest that there are very slightly linear correlations between the listed features and the overall installation cost efficiency. In the case of state_TX and self_installed, being self installed or installed in Texas both reduce the cost per KW, whereas the other factors all increase the cost per KW.

One final interesting finding was that the month of both battery and solar panel installation, as well as the year of battery installation, all have significant impact on the overall cost efficiency of the solar panel installation.

These should help me know what to watch for as I develop a model to predict how to optimize cost efficiency in the next step of this capstone.

In [ ]:
data.to_csv('processed_data.csv')